In [1]:
#GDD calculation for corn from class CornGDD 2024. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class CornGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def con_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


Using CSV values from GEMS exchange Min/Max temp 2023 and 2024 for Becker

In [4]:
import pandas as pd
Becker_GEMS = pd.read_csv("becker_gems_2023_2024.csv", on_bad_lines='skip')

In [5]:
Becker_GEMS_2024 = CornGDD(Becker_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [6]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("5-1-2024").date()
end_date = pd.to_datetime("10-17-2024").date()

Calculating Corn AGDD for Becker 2024

In [7]:
corn_gdd_BeckerGEMS = Becker_GEMS_2024.con_gdd(start_date, end_date)
print(corn_gdd_BeckerGEMS)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
486      75 2024-05-01    5.7   53  429  314      119      917      905   
487      92 2024-05-02    7.8   53  431  316      119      917      908   
488      28 2024-05-03    4.3   53  432  317      119      917      911   
489      71 2024-05-04    4.8   53  433  319      119      917      913   
490      13 2024-05-05    2.5   54  435  321      120      918      916   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
651      60 2024-10-13    1.9   36  302  163      102      838      596   
652      21 2024-10-14   -0.1   35  299  161      102      836      590   
653      18 2024-10-15   -0.8   35  297  159      102      834      584   
654       0 2024-10-16   -4.2   35  295  156      101      832      578   
655       0 2024-10-17   -3.2   34  292  154      101      830      573   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
486     62.60  ...   